In [1]:
from sentence_transformers import SentenceTransformer, util

import pandas as pd
import numpy as np

# 데이터 불러오기, 파일 다운로드 후 경로 수정 필요
baemin_df = pd.read_csv("../guides/baemin.txt", sep="\n\n", names=['text'], engine='python')
baemin_df['text'] = baemin_df['text'].apply(lambda x: '[배민] ' + x)
baemin_guide = baemin_df['text'].to_list()
kakao_df = pd.read_csv("../guides/kakao.txt", sep="\n\n", names=['text'], engine='python')
kakao_df['text'] = kakao_df['text'].apply(lambda x: '[카톡] ' + x)
kakao_guide = kakao_df['text'].to_list()

# 가이드 리스트 합치기
guide_li = baemin_guide + kakao_guide

# 실질적으로 탑케이 필요함
embedder = SentenceTransformer("BM-K/KoSimCSE-roberta-multitask")
corpus_embeddings = embedder.encode(guide_li, convert_to_tensor=True)


No sentence-transformers model found with name C:\Users\User/.cache\torch\sentence_transformers\BM-K_KoSimCSE-roberta-multitask. Creating a new one with MEAN pooling.


In [21]:

prompt = '아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. \
질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.\
\n\n### 질문 ###\n'

# Query sentences:
queries = [
    '배달의민족에서 카카오페이 결제 하는 방법 알려줘.', 
    # '카톡 사용 방법 알려줘', 
    # '치타가 들판을 가로 질러 먹이를 쫓는다.',
    # '카톡 이모티콘 무제한 어떻게 하는거야?',
    # '배민에서 결제를 어떻게 하는건지 모르겠어요'
    ]

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5

# 유사도 점수 확인
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[:top_k]
    idxs = [x.item() for x in top_results]
    prompt += query

    for num, idx in enumerate(idxs):
        if 0 <= idx < len(guide_li):
            score = float("%.2f" % cos_scores[idx])

            if score >= 0.0:
                prompt += f'\n\n### 정보{num + 1} ###\n{score}\n{guide_li[idx].strip()}'
            else:
                break
print(prompt)

아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. 질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.

### 질문 ###
배달의민족에서 카카오페이 결제 하는 방법 알려줘.

### 정보1 ###
0.81
[배민] 배민페이: 배달의 민족 앱에서 결제할 때 사용할 수 있는 서비스. 계좌나 카드 정보를 배민페이로 저장해 두면 결제할 때 연결해 쓸 수 있다.

### 정보2 ###
0.72
[배민] [배민] 다른 결제수단: 결제하기 중 다른 결제수단에 있는 '변경'을 누르면 '신용/체크카드', '휴대폰결제', 'Apple Pay', '네이버페이', '카카오페이' 등 선택하여 결제를 진행할 수 있다.

### 정보3 ###
0.71
[배민] 배달팁: 고객이 내는 배달비. 음식값에 더해져서 한 번에 결제된다.

### 정보4 ###
0.69
[배민] 가족계정 결제하기(과정): 1. 가족 구성원이 원하는 메뉴를 주문한 다음 가족 대표에게 결제 요청하기. 2. 가족 대표에게 결제 요청 알림 전달. 가족 구성원의 주문을 가족 대표가 대신 결제. 3. 가족 구성원이 음식을 배달 받음

### 정보5 ###
0.69
[카톡] 패밀리결제 이용하기: 한 사람이 대표가 되어 멤버들의 결제를 손쉽게 승인하고 관리할 수 있어요. 어린 자녀부터 나이 드신 부모님까지, 결제 수단이 없거나 결제가 어려운 지인이 있다면 패밀리결제를 시작해 보세요. 한 사람이 대표가 되어 결제를 손쉽게 진행하고, 나머지 멤버들의 결제 권한을 설정할 수도 있어요. 가족 뿐만 아니라 친구와도 패밀리결제를 신청할 수 있으니, 필요에 따라 더 폭넓게 활용할 수 있겠죠? 1. 카카오톡 설정 > 카카오계정 > 패밀리 계정 관리 메뉴를 선택하세요. 2. 패밀리 계정이 없다면 직접 대표가 되어 패밀리 계정을 생